In [16]:
import pandas as pd
import numpy as np


In [17]:
songs=pd.read_csv("songs.csv")

In [18]:
songs=songs.drop('link',axis=1).reset_index(drop=True)

In [19]:
songs

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante","Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...
...,...,...,...
57645,Ziggy Marley,Good Old Days,Irie days come on play \nLet the angels fly l...
57646,Ziggy Marley,Hand To Mouth,Power to the workers \nMore power \nPower to...
57647,Zwan,Come With Me,all you need \nis something i'll believe \nf...
57648,Zwan,Desire,northern star \nam i frightened \nwhere can ...


Preprocessing


In [20]:
songs['text']=songs['text'].str.lower().str.replace('\r', '').str.replace('\n', '')

In [21]:
# songs['text']=songs['text'].str.replace('\\r', '').str.replace('\\n', '')
songs['song']=songs['song'].str.lower()

In [22]:
songs=songs.sample(5000).reset_index(drop=True)

In [23]:
songs['recommend'] = None

In [24]:
songs

,artist,song,text,recommend
0,Cheap Trick,all i really want to do,i'm in the right place just at the wrong time ...,None
1,Glee,like a prayer,"life is a mystery, everyone must stand alone. ...",None
2,Hank Snow,"blue, blue day",it's been a blue blue day i feel like running ...,None
3,Green Day,i want to be alone,[chorus] i lock myself inside my room i want...,None
4,Crowded House,heaven that i'm making,plead with my saint wash his hands and feet ...,None
...,...,...,...,...
4995,ZZ Top,lovething,"it must have been the way that she kissed me, ...",None
4996,Nick Drake,time has told me,"time has told me you're a rare, rare find a ...",None
4997,George Harrison,absolutely sweet marie,"well, your railroad gate, you know i just can'...",None
4998,Heart,corduroy road,"zingara why, do they speak your name? whisper...",None


In [25]:
import nltk
from nltk.stem.porter import PorterStemmer

In [26]:
stemmer=PorterStemmer()

In [27]:
def token(elm):
  tok=nltk.word_tokenize(elm)
  a=[stemmer.stem(x) for x in tok]
  return ' '.join(a)

In [28]:
songs['text']=songs['text'].apply(lambda x: token(x))

In [29]:
songs['artist']=songs['artist'].str.lower()

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
tfid=TfidfVectorizer(analyzer='word')

In [32]:
matrix=tfid.fit_transform(songs['text'])

In [33]:
similarity=cosine_similarity(matrix)

In [34]:
similarity[0]

array([1.        , 0.12462726, 0.07466072, ..., 0.15518463, 0.06041591,
       0.06875993])

Recommender function

In [35]:
def recommender(song_name):
    try:
        i = songs[songs['song'] == song_name].index[0]
    except IndexError:
        # Handle the case where the song doesn't exist
        return ["Song not found. Please provide a valid song name."]

    distances = sorted(enumerate(similarity[i]), key=lambda x: x[1], reverse=True)
    song_indices = [index for index, _ in distances[1:11]]  # Exclude the input song itself

    recommended_songs = []
    for index in song_indices:
        song = songs.loc[index, 'song']
        artist = songs.loc[index, 'artist']
        recommended_songs.append(song + '   ' + artist)

    return recommended_songs


In [36]:
for index, row in songs.iterrows():
    song_name = row['song']
    recommendations = recommender(song_name)
    songs.at[index, 'recommend'] = recommendations

In [37]:
songs['recommend'].iloc[84]

['lives in the balance   jackson browne',
 'rhymes and reasons   john denver',
 'looking into you   jackson browne',
 'eldorado   neil young',
 'the house you live in   gordon lightfoot',
 'kid a   radiohead',
 '10,000 days (wings pt. 2)   tool',
 'what are you doing the rest of your life   michael bolton',
 'in the moment   uriah heep',
 'river of dreams   billy joel']

In [38]:
songs

,artist,song,text,recommend
0,cheap trick,all i really want to do,i 'm in the right place just at the wrong time...,"[easy once you know how janis joplin, it don..."
1,glee,like a prayer,"life is a mysteri , everyon must stand alon . ...","[i dreamed last night moody blues, living in..."
2,hank snow,"blue, blue day",it 's been a blue blue day i feel like run awa...,"[blue, blue day dean martin, color of the bl..."
3,green day,i want to be alone,[ choru ] i lock myself insid my room i want t...,"[leave me alone donna summer, hero ozzy os..."
4,crowded house,heaven that i'm making,plead with my saint wash hi hand and feet find...,"[i got shoes johnny cash, heaven only knows ..."
...,...,...,...,...
4995,zz top,lovething,"it must have been the way that she kiss me , m...","[ready or not jackson browne, sandra dusty..."
4996,nick drake,time has told me,"time ha told me you 're a rare , rare find a t...","[you told me the monkees, if i told you that..."
4997,george harrison,absolutely sweet marie,"well , your railroad gate , you know i just ca...","[mary, did you know? christmas songs, hold y..."
4998,heart,corduroy road,"zingara whi , do they speak your name ? whispe...","[down to the water america, honey hole lyn..."


In [39]:
import json

data_to_export = {
    "songs": songs.to_dict(orient="records")
}

with open('songsmodel.json', 'w') as file:
    json.dump(data_to_export, file)

In [40]:
# recommendation = recommender('alone')
# print(recommendation)